## 型ヒント

In [1]:
sample_int: int = 10
sample_float: float = 0.1

from typing import List, Dict

sample_list: List[int] = [1, 2, 3, 4]
sample_dict: Dict[str, str] = {"name":"Mike"}

## FastAPI Hellow World

@app.py
```python
from fastapi import FastAPI
app = FastAPI()

@app.get("/")
async def index():
    return  {"message": "Hello World"}
```

bash
```bash
uvicorn main:app --reload # uvicorn起動：localhost:8000
```
- mainはファイル名（main.py）でappはインスタンス名（app = FastAPI()）
- `localhost:8000`にアクセスすると{"message": "Hello World"}が表示。
- `localhost:8000/docs`, `localhost:8000/redoc`にアクセスすると自動生成されたドキュメント

### path parameter
- パスから変数を取得
- DBのクエリに使ったりするとデータ取り出し＆処理をして返したりできる

```python
@app.get("/countries/{country_name}")
async def country(country_name):
    return  {"country_name": country_name}
```

### query parameter
- 関数の引数のパスパラメータ以外の変数名は全てクエリパラメータとして認識される
- default値を設定しておくとクエリパラメータが無い場合に対応

```python
@app.get("/countries/")
async def country(country_name="japan", country_no=1):
    return  {
        "country_name": country_name,
        "country_no": country_no
            }
```

### 型制限
- 関数の引数に型ヒントをつけると入力される型を制限できる（間違った型が入るとエラーを返す）

```python
@app.get("/countries/{country_name}")
async def country(country_name: str):
    return  {"country_name": country_name}

@app.get("/countries/")
async def country(country_name:str="japan", country_no:int=1):
    return  {
        "country_name": country_name,
        "country_no": country_no
            }
```

### Optionalパラメータ
- typing.OptionalでnullでもOKになる
- 但しDefaultでNoneは必須

```python
from typing import Optional
@app.get("/countries/")
async def country(country_name: Optional[str]=None, country_no: Optional[int]=None):
    return  {
        "country_name": country_name,
        "country_no": country_no
            }
```

### リクエストボディ定義及びPostメソッド
- postされるjsonデータ構造をpydantic.BaseModelで定義
- 定義したクラスで型指定
- 処理して返す
- docsでリクエストとレスポンスをテストできる

```python
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    descriptiion: Optional[str] = None
    price: int
    tax: Optional[float] = None

@app.post("/item/")
async def create_item(item: Item):
    return {"message": f"{item.name}は税込価格{int(item.price*item.tax)}円です"}
```

この場合以下のようなリクエストボディを受け付ける
```json
{
    "name": "T-shirt",
    "description": "This is a White T-shirt",
    "price": 5980,
    "tax": 1.1
}
```

### requestsを使ってAPIを叩く
- urlとrequest bodyを入れる
- request bodyはjson.dumps(dict)でencodeする

```python
import requests
import json

url = "http://127.0.0.1:8000/item/"
body = {
    "name": "T-shirt",
    "description": "This is a White T-shirt",
    "price": 5980,
    "tax": 1.1
}

# json化が必要
body = json.dumps(body)

res = requests.post(url, body)
res.json()
```

- json.dumps(body)の返り値はstringのよう

```python
print(type(body))
print(body)
```

```bash
<class 'str'>
{"name": "T-shirt", "description": "This is a White T-shirt", "price": 5980, "tax": 1.1}
```

### 入子構造のデータ定義
- BaseModelをベースとしたモデルを別のモデルに入れることで定義
- typing.Listで複数入れる

```python
from typing import Optional, List
from pydantic import BaseModel

class ShopInfo(BaseModel):
    name: str
    location: str

class Item(BaseModel):
    name: str
    descriptiion: Optional[str] = None
    price: int
    tax: Optional[float] = None

class Data(BaseModel):
    shop_info: Optional[ShopInfo] = None
    items: List[Item]


@app.post("/shop_items/")
async def index(data: Data):
    return {"data": data}


### 入力データに更に制限を加える
- pydantic.Fieldクラスをデフォルト値で設定
- 入力に様々な制限を与えることができる
- デフォルト値がある場合はdefault引数

```python
from pydantic import BaseModel, Field

class Item(BaseModel):
    name: str = Field(default="t-shirt", min_length=4, max_length=12)
    descriptiion: Optional[str] = None
    price: int
    tax: Optional[float] = None 
```

### DETA cloudへのAPIデプロイ
#### 準備
- FastAPIドキュメント
https://fastapi.tiangolo.com/ja/deployment/deta/
- DETA cloudに登録（済）https://web.deta.sh/home/ishi23/default/micros
- curlでDETAをローカルインストール（済）

#### 操作
- main.pyというファイル名でのfastapiの定義ファイルとrequirements.txtを置いたフォルダ内で`data new`コマンドするだけでエンドポイントが発行される

bash

```bash
$ cd dir_containing_main_and_requirements
$ deta login # ブラウザが起動する。SafariはダメなのでChromeで。
$ data new

```

出力

```bash
Successfully created a new micro
{
        "name": "api_deploy_DETAcloud",
        "id": "2563d1a6-b126-4ca9-801c-6ddbc5aa4689",
        "project": "c0cgq90s",
        "runtime": "python3.9",
        "endpoint": "https://56vw3g.deta.dev",
        "region": "ap-southeast-1",
        "visor": "enabled",
        "http_auth": "disabled"
}
Adding dependencies...
Collecting fastapi
```

main.pyを編集したらdeta deploy

```bash
# main.pyのフォルダ
$ deta deploy
```

In [27]:
import requests
import json
import pandas as pd

url = "http://127.0.0.1:8000/organizations/"

organizations = [
    {"name": "製薬本部"},
    {"name": "製薬研究部"},
    {"name": "分析研究部"},
    {"name": "バイオ1G"},
    {"name": "バイオ2G"},
    {"name": "デジタルサイエンスG"},    
]

for body in organizations:
    body = json.dumps(body)
    res = requests.post(url, body)
    print(res.json())


{'name': '製薬本部', 'high_org': None, 'active': True, 'pk': 1}
{'name': '製薬研究部', 'high_org': None, 'active': True, 'pk': 2}
{'name': '分析研究部', 'high_org': None, 'active': True, 'pk': 3}
{'name': 'バイオ1G', 'high_org': None, 'active': True, 'pk': 4}
{'name': 'バイオ2G', 'high_org': None, 'active': True, 'pk': 5}
{'name': 'デジタルサイエンスG', 'high_org': None, 'active': True, 'pk': 6}


In [28]:
url = "http://127.0.0.1:8000/employees/"

data = [
    {"employee_id": "000001", 
     "name": "one", 
     "organization": 8, 
     "email": "one@aaa.jp"},
    {"employee_id": "000002", 
     "name": "two", 
     "organization": 8, 
     "email": "two@aaa.jp"},
    {"employee_id": "000003", 
     "name": "three", 
     "organization": 8, 
     "email": "three@aaa.jp"},
    {"employee_id": "000004", 
     "name": "four", 
     "organization": 9, 
     "email": "four@aaa.jp"},
    {"employee_id": "000005", 
     "name": "five", 
     "organization": 9, 
     "email": "five@aaa.jp"},
]

for body in data:
    body = json.dumps(body)
    res = requests.post(url, body)
    print(res.json())


{'employee_id': '000001', 'name': 'one', 'organization': 8, 'email': 'one@aaa.jp', 'active': True, 'pk': 1}
{'employee_id': '000002', 'name': 'two', 'organization': 8, 'email': 'two@aaa.jp', 'active': True, 'pk': 2}
{'employee_id': '000003', 'name': 'three', 'organization': 8, 'email': 'three@aaa.jp', 'active': True, 'pk': 3}
{'employee_id': '000004', 'name': 'four', 'organization': 9, 'email': 'four@aaa.jp', 'active': True, 'pk': 4}
{'employee_id': '000005', 'name': 'five', 'organization': 9, 'email': 'five@aaa.jp', 'active': True, 'pk': 5}


In [29]:
url = "http://127.0.0.1:8000/themes/"

data = [
    {"name": "themeA"},
    {"name": "themeB"},
    {"name": "themeC"},
    {"name": "themeD"},
    {"name": "themeE"},
    {"name": "themeF"},    
]

for body in data:
    body = json.dumps(body)
    res = requests.post(url, body)
    print(res.json())


{'name': 'themeA', 'active': True, 'pk': 1}
{'name': 'themeB', 'active': True, 'pk': 2}
{'name': 'themeC', 'active': True, 'pk': 3}
{'name': 'themeD', 'active': True, 'pk': 4}
{'name': 'themeE', 'active': True, 'pk': 5}
{'name': 'themeF', 'active': True, 'pk': 6}


In [30]:
url = "http://127.0.0.1:8000/kwcategories/"

data = [
    {"name": "cateA"},
    {"name": "cateB"},
    {"name": "cateC"},
    {"name": "cateD"},
    {"name": "cateE"},
    {"name": "cateF"},    
]

for body in data:
    body = json.dumps(body)
    res = requests.post(url, body)
    print(res.json())


{'name': 'cateA', 'active': True, 'pk': 1}
{'name': 'cateB', 'active': True, 'pk': 2}
{'name': 'cateC', 'active': True, 'pk': 3}
{'name': 'cateD', 'active': True, 'pk': 4}
{'name': 'cateE', 'active': True, 'pk': 5}
{'name': 'cateF', 'active': True, 'pk': 6}


In [31]:
url = "http://127.0.0.1:8000/keywords/"

data = [
    {"name": "keyA", "category": 1},
    {"name": "keyB", "category": 2},
    {"name": "keyC", "category": 3},
    {"name": "keyD", "category": 3},
    {"name": "keyE", "category": 2},
    {"name": "keyF", "category": 4},    
]

for body in data:
    body = json.dumps(body)
    res = requests.post(url, body)
    print(res.json())



{'name': 'keyA', 'category': 1, 'active': True, 'pk': 1}
{'name': 'keyB', 'category': 2, 'active': True, 'pk': 2}
{'name': 'keyC', 'category': 3, 'active': True, 'pk': 3}
{'name': 'keyD', 'category': 3, 'active': True, 'pk': 4}
{'name': 'keyE', 'category': 2, 'active': True, 'pk': 5}
{'name': 'keyF', 'category': 4, 'active': True, 'pk': 6}


In [33]:
url = "http://127.0.0.1:8000/meetings/"

data = [
    {"name": "meetingA",
     "description": "something", 
     "members": [1,2,3], 
     "responsible_person": 1, 
     "host_org": 1,
     "periodic": True,
    },
    {"name": "meetingB",
     "description": "something", 
     "members": [1,2,3], 
     "responsible_person": 1, 
     "host_org": 1,
     "periodic": True,
    },
    {"name": "meetingC",
     "description": "something", 
     "members": [1,2,3], 
     "responsible_person": 1, 
     "host_org": 1,
     "periodic": True,
    },
    {"name": "meetingD",
     "description": "something", 
     "members": [1,2,3], 
     "responsible_person": 1, 
     "host_org": 1,
     "periodic": True,
    },
    {"name": "meetingE",
     "description": "something", 
     "members": [1,2,3], 
     "responsible_person": 1, 
     "host_org": 1,
     "periodic": True,
    },
    {"name": "meetingF",
     "description": "something", 
     "members": [1,2,3], 
     "responsible_person": 1, 
     "host_org": 1,
     "periodic": True,
    },    
]

for body in data:
    body = json.dumps(body)
    res = requests.post(url, body)
    print(res.json())



JSONDecodeError: [Errno Expecting value] Internal Server Error: 0